# Deep Learning aplicado a un guión de película

## Importación del guión a Df

In [1]:
import pandas as pd
import re

In [2]:
# Cargar el archivo de subtítulos
# Probé con: utf-8
ruta_archivo = "Jerry.Maguire.1996.Blu-ray.1080p.x264.HDBRiSe.SPA.srt"
with open(ruta_archivo, "r", encoding="latin-1") as f:
    contenido = f.read()

In [3]:
# Dividir el contenido en bloques separados por líneas en blanco
bloques = contenido.strip().split("\n\n")

# Listas para almacenar los datos
inicio = []
fin = []
subtitulos = []

# Procesar cada bloque
for bloque in bloques:
    lineas = bloque.split("\n")
    if len(lineas) < 3:
        continue  # Saltar bloques incompletos

    # Extraer el rango de tiempo
    tiempo = lineas[1]
    tiempo_match = re.match(r"(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})", tiempo)
    if tiempo_match:
        inicio.append(tiempo_match.group(1))
        fin.append(tiempo_match.group(2))
    else:
        inicio.append(None)
        fin.append(None)

    # Unir las líneas de texto del subtítulo
    texto = " ".join(lineas[2:]).strip()
    # Eliminar etiquetas como <i> o </i>
    texto_limpio = re.sub(r"<.*?>", "", texto)
    subtitulos.append(texto_limpio)

# Crear el DataFrame
df = pd.DataFrame({
    "inicio": inicio,
    "fin": fin,
    "subtitulo": subtitulos
})

In [4]:
df.to_csv("subtitulos.csv", index=False, encoding="utf-8")

In [5]:
df.head()

,inicio,fin,subtitulo
0,"00:00:42,870","00:00:47,041",El mundo. Casi 6 mil millones de personas.
1,"00:00:47,291","00:00:51,671",Cuando era niño había tres mil. Cuesta estar a...
2,"00:00:53,340","00:00:56,218",Así está mejor. Esto es América.
3,"00:00:56,468","00:00:59,763",América sigue marcando los cánones del mundo.
4,"00:01:01,890","00:01:05,811","En Indiana, Clark Hodd. El mejor ala pivot del..."


In [6]:
df.shape

(1486, 3)

## NLP

In [7]:
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
import string

In [8]:
# Cargar modelo de español
nlp = spacy.load("es_core_news_sm")

In [9]:
# Función para limpiar el texto: convertir a minúsculas y eliminar puntuación
def limpiar_texto(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

In [10]:
# Aplicar limpieza al subtítulo
df['subtituloLimpio'] = df['subtitulo'].apply(limpiar_texto)

In [11]:
# Tokenizar cada subtítulo usando SpaCy
df["tokens"] = df["subtituloLimpio"].apply(lambda text: [token.text for token in nlp(text)])

In [12]:
df[["subtitulo","subtituloLimpio", "tokens"]].sample(5)

,subtitulo,subtituloLimpio,tokens
368,"- Repítelo conmigo, Jerry. - Te enseño el dinero.",repítelo conmigo jerry te enseño el dinero,"[ , repítelo, conmigo, jerry, , te, enseño, e..."
531,Puede que incluso funcione.,puede que incluso funcione,"[puede, que, incluso, funcione]"
808,"Dennis, te estoy pidiendo...",dennis te estoy pidiendo,"[dennis, te, estoy, pidiendo]"
1441,Te quiero.,te quiero,"[te, quiero]"
590,"Bueno, oye, es la novatada. Yo arreglaré lo de...",bueno oye es la novatada yo arreglaré lo de de...,"[bueno, oye, es, la, novatada, yo, arreglaré, ..."


In [13]:
# Eliminar stopwords de los tokens
df["tokensSinStopwords"] = df["tokens"].apply(lambda tokens: [token for token in tokens if not nlp.vocab[token].is_stop])
# df['tokens_sin_stopwords'] = df['tokens'].apply(lambda tokens: [token for token in tokens if token not in STOP_WORDS])

In [ ]:
df[["subtitulo", "tokens","tokensSinStopwords"]].sample(5)

,subtitulo,tokens,tokensSinStopwords
862,"No estamos peleando, por una vez estamos habla...","[no, estamos, peleando, por, una, vez, estamos...","[peleando, hablando]"
144,Así que me volví hacia ella y le dije:,"[así, que, me, volví, hacia, ella, y, le, dije]","[volví, dije]"
170,- Dorothy...,"[ , dorothy]","[ , dorothy]"
749,"Has dicho ""puto"".","[has, dicho, puto]","[has, puto]"
491,Coge el avión. ¿De acuerdo?,"[coge, el, avión, ¿, de, acuerdo]","[coge, avión, ¿]"


In [15]:
# Lematizar los tokens (eliminar stopwords primero si es necesario)
df["tokensLematizados"] = df["tokensSinStopwords"].apply(lambda tokens: [nlp(token)[0].lemma_ for token in tokens])

In [16]:
df[["subtitulo", "tokens","tokensSinStopwords","tokensLematizados"]].sample(5)

,subtitulo,tokens,tokensSinStopwords,tokensLematizados
968,Estocolmo 1963.,"[estocolmo, 1963]","[estocolmo, 1963]","[estocolmo, 1963]"
1067,- Lo haré. - No necesitamos un camello.,"[ , lo, haré, , no, necesitamos, un, camello]","[ , haré, , necesitamos, camello]","[ , hacer, , necesitar, camello]"
468,No lo puedo evitar.,"[no, lo, puedo, evitar]",[evitar],[evitar]
1099,- Casi hasta me cae bien el chico. - Adiós.,"[ , casi, hasta, me, cae, bien, el, chico, , ...","[ , cae, chico, , adiós]","[ , caer, chico, , adiós]"
1139,"Oye, ése es mi hermano.","[oye, ése, es, mi, hermano]","[oye, hermano]","[oir, hermano]"


## Análisis de Sentimiento

In [17]:
from textblob import TextBlob

# Función para obtener la polaridad del sentimiento
def sentimientoTextblob(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # Devuelve el puntaje de polaridad (-1 a 1)

In [18]:
# Aplicar el análisis de sentimiento a los subtítulos originales
df['sentimiento'] = df['subtitulo'].apply(sentimientoTextblob)

In [19]:
df[["subtitulo", "sentimiento"]].sample(5)

,subtitulo,sentimiento
136,Decidí no declararme. No estaba de humor.,0.0
1190,Mierda.,0.0
1033,...que ya no sé qué decirle a mi hijo.,0.0
449,Cush todo lo consigue.,0.0
1343,Tidwell. Gol de Cardinals.,0.0


In [20]:
df[["sentimiento"]].describe()

,sentimiento
count,1486.000000
mean,-0.000585
std,0.055696
min,-1.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.600000


Me parece demasiado neutral el análisis de sentimiento.

In [21]:
from textblob import TextBlob

def sentimiento_frases(text):
    sentences = TextBlob(text).sentences
    return sum(sentence.sentiment.polarity for sentence in sentences) / len(sentences)

df['sentimientoSentence'] = df['subtitulo'].apply(sentimiento_frases)
df[['sentimientoSentence','subtitulo']].sample(5)

MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.
